In [1]:
import pandas as pd
data = pd.read_csv("flight_data_2021.csv")

In [2]:
columns_to_drop = ['Origin','Dest','DestState','OriginState',
                   'Operated_or_Branded_Code_Share_Partners',
                   'arr-type','arr-elevation_ft']
data = data.drop(columns_to_drop, axis=1)
data.head()

,Airline,Cancelled,Quarter,Month,DayOfWeek,DepDelayMinutes,DepTimeBlk,dept-type,dept-elevation_ft,Route_Popularity,Distance_Final,Wind_Speed_mph,Wind_Gust_mph,Visibility_miles,tempF,precip_in,daily_snow_in
0,SkyWest Airlines Inc.,False,1,3,3,0.0,0700-0759,medium_airport,2941.0,937.0,422.246813,10.0,17.0,6.0,49,0.0,0.0
1,SkyWest Airlines Inc.,False,1,3,3,0.0,0900-0959,large_airport,1135.0,937.0,422.246813,6.0,9.0,6.0,60,0.0,0.0
2,SkyWest Airlines Inc.,False,1,3,3,0.0,1300-1359,large_airport,266.0,918.0,1356.657710,10.0,18.0,6.0,39,0.0,0.0
3,SkyWest Airlines Inc.,False,1,3,3,0.0,1600-1659,large_airport,607.0,568.0,1396.572313,10.0,16.0,6.0,65,0.0,0.0
4,SkyWest Airlines Inc.,False,1,3,3,0.0,1800-1859,large_airport,1135.0,1135.0,684.252343,21.0,35.0,6.0,71,0.0,0.0


In [20]:
noncancelled_data = data[data['Cancelled']==False]

In [21]:
regression_df=noncancelled_data
# Drop the 'Cancelled' column
regression_df.drop('Cancelled', axis=1, inplace=True)

regression_df.head()

/tmp/ipykernel_1206078/610716977.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regression_df.drop('Cancelled', axis=1, inplace=True)


,Airline,Quarter,Month,DayOfWeek,DepDelayMinutes,DepTimeBlk,dept-type,dept-elevation_ft,Route_Popularity,Distance_Final,Wind_Speed_mph,Wind_Gust_mph,Visibility_miles,tempF,precip_in,daily_snow_in
0,SkyWest Airlines Inc.,1,3,3,0.0,0700-0759,medium_airport,2941.0,937.0,422.246813,10.0,17.0,6.0,49,0.0,0.0
1,SkyWest Airlines Inc.,1,3,3,0.0,0900-0959,large_airport,1135.0,937.0,422.246813,6.0,9.0,6.0,60,0.0,0.0
2,SkyWest Airlines Inc.,1,3,3,0.0,1300-1359,large_airport,266.0,918.0,1356.657710,10.0,18.0,6.0,39,0.0,0.0
3,SkyWest Airlines Inc.,1,3,3,0.0,1600-1659,large_airport,607.0,568.0,1396.572313,10.0,16.0,6.0,65,0.0,0.0
4,SkyWest Airlines Inc.,1,3,3,0.0,1800-1859,large_airport,1135.0,1135.0,684.252343,21.0,35.0,6.0,71,0.0,0.0


In [22]:
regression_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5861080 entries, 0 to 5967955
Data columns (total 16 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Airline            object 
 1   Quarter            int64  
 2   Month              int64  
 3   DayOfWeek          int64  
 4   DepDelayMinutes    float64
 5   DepTimeBlk         object 
 6   dept-type          object 
 7   dept-elevation_ft  float64
 8   Route_Popularity   float64
 9   Distance_Final     float64
 10  Wind_Speed_mph     float64
 11  Wind_Gust_mph      float64
 12  Visibility_miles   float64
 13  tempF              int64  
 14  precip_in          float64
 15  daily_snow_in      float64
dtypes: float64(9), int64(4), object(3)
memory usage: 760.2+ MB


In [23]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Load your data
# regression_df = pd.read_csv('path_to_your_data.csv')

# Define categorical and numerical features
categorical_features = ['Airline', 'DepTimeBlk', 'dept-type']
numerical_features = ['Quarter', 'Month', 'DayOfWeek', 'dept-elevation_ft', 'Route_Popularity',
                      'Distance_Final', 'Wind_Speed_mph', 'Wind_Gust_mph', 'Visibility_miles', 
                      'tempF', 'precip_in', 'daily_snow_in']

# One-hot encode categorical variables
encoder = OneHotEncoder(sparse=False)
regression_df_categorical_encoded = pd.DataFrame(encoder.fit_transform(regression_df[categorical_features]), 
                                                 columns=encoder.get_feature_names_out(),
                                                 index=regression_df.index)  # Maintain the same index

# Concatenate encoded variables back to the DataFrame and drop original categorical columns
regression_df = pd.concat([regression_df.drop(categorical_features, axis=1), regression_df_categorical_encoded], axis=1)

# Normalize the dataset
scaler = MinMaxScaler()
regression_df[numerical_features] = scaler.fit_transform(regression_df[numerical_features])
scaled_target = np.log1p(regression_df['DepDelayMinutes'].values.reshape(-1, 1))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(regression_df[numerical_features + list(encoder.get_feature_names_out())], 
                                                    scaled_target, test_size=0.2, random_state=42)

# Convert arrays to PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Reshape data for LSTM [samples, time steps, features]
X_train = X_train.unsqueeze(1)
X_test = X_test.unsqueeze(1)

# Create DataLoader instances
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=64)


In [25]:
# This will give you the count of NaN values per column
nan_counts = regression_df.isna().sum()
print(nan_counts)


Quarter                                              0
Month                                                0
DayOfWeek                                            0
DepDelayMinutes                                      0
dept-elevation_ft                                    0
Route_Popularity                                     0
Distance_Final                                       0
Wind_Speed_mph                                       0
Wind_Gust_mph                                        0
Visibility_miles                                     0
tempF                                                0
precip_in                                            0
daily_snow_in                                        0
Airline_Air Wisconsin Airlines Corp                  0
Airline_Alaska Airlines Inc.                         0
Airline_Allegiant Air                                0
Airline_American Airlines Inc.                       0
Airline_Capital Cargo International                  0
Airline_Co

In [26]:
# Define the LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        h0, c0 = (torch.zeros(1, x.size(0), hidden_dim), 
                  torch.zeros(1, x.size(0), hidden_dim))
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [28]:
# Model initialization
input_dim = X_train.shape[2]
hidden_dim = 50
num_layers = 1
output_dim = 1

model = LSTMModel(input_dim, hidden_dim, num_layers, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training the Model
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

train_model(model, train_loader, criterion, optimizer)

# Evaluate the Model
def evaluate_model(model, test_loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    print(f'Test Loss: {total_loss / len(test_loader)}')

evaluate_model(model, test_loader)

Epoch 1/10, Loss: 1.9614464044570923
Epoch 2/10, Loss: 2.0787110328674316
Epoch 3/10, Loss: 2.1460819244384766
Epoch 4/10, Loss: 3.2348787784576416
Epoch 5/10, Loss: 1.915832281112671
Epoch 6/10, Loss: 2.9343841075897217
Epoch 7/10, Loss: 1.481782078742981
Epoch 8/10, Loss: 1.4127033948898315
Epoch 9/10, Loss: 1.7883566617965698
Epoch 10/10, Loss: 1.7714084386825562
Test Loss: 2.0095396807625137


In [33]:
import numpy as np
import torch
from sklearn.metrics import mean_squared_error, r2_score
def evaluate_model_2(model, test_loader):
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            predictions.append(outputs.numpy())
            actuals.append(labels.numpy())
    
    # Flatten the lists
    predictions = np.concatenate(predictions, axis=0)
    actuals = np.concatenate(actuals, axis=0)
    
    # Calculate RMSE
    rmse = mean_squared_error(actuals, predictions, squared=False)
    print(f'Root Mean Squared Error: {rmse}')
    
    # Calculate R-squared
    r2 = r2_score(actuals, predictions)
    print(f'R-squared: {r2}')

    return rmse, r2

In [34]:
rmse, r_squared = evaluate_model_2(model, test_loader)

Root Mean Squared Error: 1.4175834655761719
R-squared: 0.14041593633863647
